# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#conda install -c conda-forge gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from ..api_keys import g_key

ImportError: attempted relative import with no known parent package

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
city_df = pd.read_csv(r'/Users/eline/Documents/API-Challenge/WeatherPy/city_df.csv')
city_df.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Latitude and longitude as locations, humidity as weight
locations = city_df[["Latitude", "Longitude"]].astype(float)
humidity = city_df["Humidity"].astype(float)



# Add Heatmap
display = gmaps.figure(map_type = "HYBRID")
heatmap = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 5)
display.add_layer(heatmap)
display

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
vacationweather = city_df.loc[(city_df["Max Temperature"] >= 70) & (city_df["Max Temperature"] <= 80) & (city_df["Windspeed"] <+ 10)]
vacationweather

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = vacationweather
hotel_df.reset_index(drop = True, inplace = True)
hotel_df

In [ ]:
hotel_df["Hotel Name"] = ''

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 50000,
    "rankby": "prominence",
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key,
}

# Use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # Declare variables from df
    lat = row['Latitude']
    long = row['Longitude']
    city = row['City']
    params["location"] = f'{lat},{long}'
        
    # Assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response['results']
    
    try:
        print(f"Closest {city} hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except:
        print(f"Missing field/result for {city}... skipping.")
        
    print("------------")

In [ ]:
hotel_df

In [ ]:
# Change names 
hotel_df = hotel_df.rename(columns={"Latitude": "Lat", "Longitude": "Lng"})

# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
display.add_layer(markers)

# Display figure
display